In [1]:
import pybamm as pb
import pandas as pd
import numpy as np

In [2]:
chemistry={
    "chemistry": "lithium-ion",
    "cell": "A123_Lain2019",
    "anode": "graphite_Chen2020",
    "separator": "separator_Chen2020",
    "cathode": "LFP_Prada2013",
    "electrolyte": "lipf6_Nyman2008",
    "experiment": "4C_discharge_from_full_Prada2013",
    "sei": "ramadass2004",
    "citation": ["Chen2020", "lain2019design", "prada2013simplified"],
}

In [3]:
options = {"sei": "ec reaction limited", "sei porosity change": True}
param = pb.ParameterValues(chemistry=chemistry)
model = pb.lithium_ion.DFN(options)

In [4]:
# Import drive cycle from file
drive_cycle = pd.read_csv("/home/ubuntu/CA.csv", comment="#", header=None).to_numpy()

# Create interpolant
timescale = param.evaluate(model.timescale)
current_interpolant = pb.Interpolant(drive_cycle, timescale * pb.t)

# Set drive cycle
param["Current function [A]"] = current_interpolant

In [5]:
sim = pb.Simulation(model, parameter_values=param)

In [6]:
solver=pb.CasadiSolver(mode="safe", dt_max=1)

In [7]:
sim.solve(solver=solver)

In [8]:
sim.plot(
    [
        "Current [A]",
        "Total current density [A.m-2]",
        "Terminal voltage [V]",
        "Discharge capacity [A.h]",
        "Loss of lithium to negative electrode sei [mol]"
    ]
)

interactive(children=(FloatSlider(value=0.0, description='t', max=192.75842367225732, step=1.9275842367225733)…

In [12]:
sim.plot(["Discharge capacity [A.h]"])

interactive(children=(FloatSlider(value=0.0, description='t', max=192.75842367225732, step=1.9275842367225733)…

In [ ]:
sol = sim.solution
sol.save_data("sol_data_T7.csv", ["Current [A]", "Terminal voltage [V]", "Discharge capacity [A.h]", "Loss of lithium to negative electrode sei [mol]"], to_format="csv")

### Try Multiple cycles with Modulo operator

In [11]:
# Import drive cycle from file
drive_cycle = pd.read_csv("/home/ubuntu/CA.csv", comment="#", header=None).to_numpy()

cycle_time = drive_cycle.shape[0] - 1
print(cycle_time)

# Create interpolant
timescale = param.evaluate(model.timescale)
current_interpolant = pb.Interpolant(drive_cycle, (timescale * pb.t) % cycle_time)

# Set drive cycle
param["Current function [A]"] = current_interpolant

1909


In [14]:
solver=pb.CasadiSolver(mode="safe", dt_max = 1)
sim = pb.Simulation(model, parameter_values=param, solver=solver)
sim_days = 7

In [15]:
sim.solve([0, cycle_time * sim_days])

psetup failed: .../casadi/interfaces/sundials/idas_interface.cpp:849: Calculating Jacobian failed
psetup failed: .../casadi/interfaces/sundials/idas_interface.cpp:849: Calculating Jacobian failed
psetup failed: .../casadi/interfaces/sundials/idas_interface.cpp:849: Calculating Jacobian failed
psetup failed: .../casadi/interfaces/sundials/idas_interface.cpp:849: Calculating Jacobian failed
psetup failed: .../casadi/interfaces/sundials/idas_interface.cpp:849: Calculating Jacobian failed
psetup failed: .../casadi/interfaces/sundials/idas_interface.cpp:849: Calculating Jacobian failed
psetup failed: .../casadi/interfaces/sundials/idas_interface.cpp:849: Calculating Jacobian failed
psetup failed: .../casadi/interfaces/sundials/idas_interface.cpp:849: Calculating Jacobian failed
psetup failed: .../casadi/interfaces/sundials/idas_interface.cpp:849: Calculating Jacobian failed
psetup failed: .../casadi/interfaces/sundials/idas_interface.cpp:849: Calculating Jacobian failed
psetup failed: .../c

In [16]:
# use this instead to get rid of the warning
t_drive = drive_cycle[:, 0]
step_size = np.min(np.diff(t_drive))
t_eval = np.arange(0, cycle_time * sim_days, step_size)
sim.solve(t_eval)
# you can play with step_size to change the level of resolution of the drive cycle

In [17]:
sim.plot(
    [
        "Current [A]",
        "Total current density [A.m-2]",
        "Terminal voltage [V]",
        "Discharge capacity [A.h]",
        "Loss of lithium to negative electrode sei [mol]",
        "Local ECM resistance [Ohm]"
    ]
)

interactive(children=(FloatSlider(value=0.0, description='t', max=192.75842367225732, step=1.9275842367225733)…